# Reducer Overview

Reducers are the way to aggregate data over time, space, bands, arrays and other data structures in Earth Engine. The `ee.Reducer` class specifies how data is aggregated. The reducers in this class can specify a simple statistic to use for the aggregation (e.g. minimum, maximum, mean, median, standard deviation, etc.), or a more complex summary of the input data (e.g. histogram, linear regression, list). Reductions may occur over:

- **time**  = `imageCollection.reduce()`
- **space** = `image.reduceRegion()` and `image.reduceNeighborhood()`**
- **bands**  = `image.reduce()`,

Or the attribute space of a `FeatureCollection` (`featureCollection.reduceColumns()` or `FeatureCollection` methods that start with `aggregate_`).


## Reducers have inputs and outputs

Reducers take an input dataset and produce a single output. When a single input reducer is applied to a multi-band image, Earth Engine automatically replicates the reducer and applies it separately to each band. As a result, the output image has the same number of bands as the input image; each band in the output is the reduction of pixels from the corresponding band in the input data. Some reducers take tuples of input datasets. These reducers will not be automatically replicated for each band. For example, `ee.Reducer.LinearRegression()` takes multiple predictor datasets (representing independent variables in the regression) in a particular order (see Regression reducers section bellow).

Some reducers produce multiple outputs, for example `ee.Reducer.minMax()`, `ee.Reducer.histogram()` or `ee.Reducer.toList()`. For example:

In [ ]:
# Load and filter the Sentinel-2 image collection.
collection = ee.ImageCollection('COPERNICUS/S2')\
               .filterDate('2016-01-01', '2016-12-31')\
               .filterBounds(ee.Geometry.Point([-81.31, 29.90]))

# Reduce the collection.
extrema = collection.reduce(ee.Reducer.minMax())

This will produce an output with twice the number of bands of the inputs, where band names in the output have ‘_min’ or ‘_max’ appended to the band name.

The output type should match the computation. For example, a reducer applied to an ImageCollection has an Image output. Because the output is interpreted as a pixel value, you must use reducers with a numeric output to reduce an ImageCollection (reducers like `toList()` or `histogram()` won’t work).

## Reducers use weighted inputs

By default, reductions over pixel values are weighted by their mask, though this behavior can be changed (see the Weighting section bellow). Pixels with mask equal to 0 will not be used in the reduction.

## Combining reducers

If your intent is to apply multiple reducers to the same inputs, it's good practice to `combine()` the reducers for efficiency. Specifically, calling `combine()` on a reducer with `sharedInputs` set to `true` will result in only a single pass over the data. For example, to compute the mean and standard deviation of pixels in an image, you could use something like this:

In [ ]:
from pprint import pprint 

# Load a Landsat 8 image.
image = ee.Image('LANDSAT/LC08/C01/T1/LC08_044034_20140318')

# Combine the mean and standard deviation reducers.
reducers = ee.Reducer.mean().combine(
  reducer2=ee.Reducer.stdDev(),
  sharedInputs=True
)

# Use the combined reducer to get the mean and SD of the image.
stats = image.reduceRegion(
  reducer=reducers,
  bestEffort=True,
)

# Display the dictionary of band means and SDs.
pprint(stats.getInfo())

In the output, note that the names of the reducers have been appended to the names of the inputs to distinguish the reducer outputs. This behavior also applies to image outputs, which will have the name of the reducer appended to output band names.

# ImageCollection Reductions

Consider the example of needing to take the median over a time series of images represented by an `ImageCollection`. To reduce an `ImageCollection`, use `imageCollection.reduce()`. This reduces the collection of images to an individual image as illustrated in Figure 1. Specifically, the output is computed pixel-wise, such that each pixel in the output is composed of the median value of all the images in the collection at that location. To get other statistics, such as mean, sum, variance, an arbitrary percentile, etc., the appropriate reducer should be selected and applied. For basic statistics like min, max, mean, etc., `ImageCollection` has shortcut methods like `min()`, `max()`, `mean()`, etc. They function in exactly the same way as calling reduce(), except the resultant band names will not have the name of the reducer appended.


<center>
<image src = "https://developers.google.com/earth-engine/images/Reduce_ImageCollection.png">  
</center>

<center>
  Figure 1. Illustration of an ee.Reducer applied to an ImageCollection.

In [ ]:
# Load an image collection, filtered so it's not too much data.
collection = ee.ImageCollection('LANDSAT/LT05/C01/T1')\
               .filterDate('2008-01-01', '2008-12-31')\
               .filter(ee.Filter.eq('WRS_PATH', 44))\
               .filter(ee.Filter.eq('WRS_ROW', 34))

# Compute the median in each band, each pixel.
# Band names are B1_median, B2_median, etc.
median = collection.reduce(ee.Reducer.median())

# The output is an Image.  Add it to the map.
vis_param = {'bands': ['B4_median', 'B3_median', 'B2_median'], 'gamma': 1.6}
median_tk = median.getMapId(vis_param)

center = [37.7924, -122.3355]
Mapdisplay(center,{'Landsat 5':median_tk},zoom_start=9)

This returns a multi-band Image, each pixel of which is the median of all unmasked pixels in the ImageCollection at that pixel location. Specifically, the reducer has been repeated for each band of the input imagery. Note that the band names have the name of the reducer appended: ‘B1_median’, ‘B2_median’, etc.


# Image Reductions

To reduce an `Image`, use `image.reduce()`. Reducing an image functions in an analogous way to `imageCollection.reduce()`, except the bands of the image are input to the reducer rather than the images in the collection. The output is also an image with number of bands equal to number of reducer outputs. For example:




In [ ]:
# Load an image and select some bands of interest.
image = ee.Image('LANDSAT/LC08/C01/T1/LC08_044034_20140318')\
          .select(['B4', 'B3', 'B2'])

# Reduce the image to get a one-band maximum value image.
maxValue = image.reduce(ee.Reducer.max())

# Display the result
vis_param = {'max': 13000, 'gamma': 1.6}
center = [37.7924, -122.3355]

maxValue_tk = maxValue.getMapId(vis_param)
Mapdisplay(center,{'Landsat 5':maxValue_tk},zoom_start=10)

# Statistics of an Image Region

Suppose there is need to calculate statistics over a region (or regions) of an `ee.Image`. To get statistics of pixel values in an image region, use `image.reduceRegion()`. This reduces all the pixels in the region(s) to a statistic or other compact representation of the pixel data in the region (e.g. histogram). The region is represented as a `Geometry`, which might be a polygon, containing many pixels, or it might be a single point, in which case there will only be one pixel in the region. In either case, as illustrated in Figure 2,  the output is a statistic derived from the pixels in the region.

<center>
<image src=" https://developers.google.com/earth-engine/images/Reduce_region_diagram.png" >
</center>
  

<center>
  Figure 2. An illustration of an `ee.Reducer` applied to an image and a region.
</center>
  
  
For an example of getting pixel statistics in a region of an image using reduceRegion(), consider finding the mean spectral values of a 5-year Landsat composite within the boundaries of the Sierra Nevada Coniferous Forest:
  
  

In [ ]:
# Load input imagery: Landsat 7 5-year composite.
image = ee.Image('LANDSAT/LE7_TOA_5YEAR/2008_2012')

# Load an input region: Sierra Nevada mixed conifer forest.
region = ee.Feature(ee.FeatureCollection(
  'ft:1Ec8IWsP8asxN-ywSqgXWMuBaxI6pPaeh6hC64lA')
  .filter(ee.Filter.eq('G200_REGIO', 'Sierra Nevada Coniferous Forests'))\
  .first())

# Reduce the region. The region parameter is the Feature geometry.
meanDictionary = image.reduceRegion(**{
  'reducer': ee.Reducer.mean(),
  'geometry': region.geometry(),
  'scale': 30,
  'maxPixels': 1e9
})

# The result is a Dictionary.  Print it.
pprint(meanDictionary.getInfo())

Note that in this example the reduction is specified by providing the `reducer` (`ee.Reducer.mean()`), the `geometry` (`region.geometry()`), the `scale` (30 meters) and `maxPixels` for the maximum number of pixels to input to the reducer. A scale should always be specified in reduceRegion() calls. This is because in complex processing flows, which may involve data from different sources with different scales, the scale of the output will not be unambiguously determined from the inputs. In that case, the scale defaults to 1 degree, which generally produces unsatisfactory results. See [this page](https://developers.google.com/earth-engine/scale) for more information about how Earth Engine handles scale.

There are two ways to set the scale: by specifying the scale parameter, or by specifying a CRS and CRS transform. (See the [glossary](https://developers.google.com/earth-engine/glossary) for more information about CRS's and CRS transforms). For example, the meanDictionary reduction (above) is equivalent to the following:

In [ ]:
# As an alternative to specifying scale, specify a CRS and a CRS transform.
# Make this array by constructing a 4326 projection at 30 meters,
# then copying the bounds of the composite, from composite.projection().
affine = [0.00026949458523585647, 0, -180, 0, -0.00026949458523585647, 86.0000269494563];

# Perform the reduction, print the result.
pprint(image.reduceRegion(**{
  'reducer': ee.Reducer.mean(),
  'geometry': region.geometry(),
  'crs': 'EPSG:4326',
  'crsTransform': affine,
  'maxPixels': 1e9
}).getInfo()) 

In general, specifying the scale is sufficient and results in more readable code. Earth Engine determines which pixels to input to the reducer by first rasterizing the region. If a scale is specified without a CRS, the region is rasterized in the image's native projection scaled to the specified resolution. If both a CRS and scale are specified, the region is rasterized based on them. Pixels are ‘in’ the region if their centroid is covered by the region at the specified scale and projection.

The `maxPixels` parameter is needed to get the computation to succeed. If this parameter is left out of the example, an error is returned, which looks something like:



```
Dictionary (Error)
  Image.reduceRegion: Too many pixels in the region. Found 527001545, but only 10000000 allowed.
```


There are multiple options to get past these errors: increase maxPixels, as in the example, increase the scale, or set bestEffort to true, which automatically computes a new (larger) scale such that maxPixels is not exceeded. If you do not specify maxPixels, the default value is used.

# Statistics of Image Neighborhoods

Rather than specifying a region over which to perform a reduction, it is also possible to specify a neighborhood in which to apply a reducer. To reduce image neighborhoods, use `image.reduceNeighborhood()`. In this case, the reduction will occur in a sliding window over the input image, with the window size and shape specified by an `ee.Kernel`. The output of `reduceNeighborhood()` will be another image, with each pixel value representing the output of the reduction in a neighborhood around that pixel in the input image. Figure 3 illustrates this type of reduction.

<center>
<img src="https://developers.google.com/earth-engine/images/Reduce_Neighborhood.png">  
</center>  

<center>
Figure 3. Illustration of `reduceNeighborhood()`, where the reducer is applied in a kernel.
</center>  


For example, consider using National Agriculture Imagery Program (NAIP) imagery to quantify landscape differences resulting from logging in the California redwood forests. Specifically, use standard deviation (SD) in a neighborhood to represent the difference in texture between the logged area and the protected area. For example, to get texture of a NAIP Normalized Difference Vegetation Index (NDVI) image, use reduceNeighborhood() to compute SD in a neighborhood defined by a kernel:



In [ ]:
# Define a region in the redwood forest.
redwoods = ee.Geometry.Rectangle(-124.0665, 41.0739, -123.934, 41.2029)

# Load input NAIP imagery and build a mosaic.
naipCollection = ee.ImageCollection('USDA/NAIP/DOQQ')\
                   .filterBounds(redwoods)\
                   .filterDate('2012-01-01', '2012-12-31')

naip = naipCollection.mosaic()

# Compute NDVI from the NAIP imagery.
naipNDVI = naip.normalizedDifference(['N', 'R'])

# Compute standard deviation (SD) as texture of the NDVI.
texture = naipNDVI.reduceNeighborhood(**{
  'reducer': ee.Reducer.stdDev(),
  'kernel': ee.Kernel.circle(7),
})

# Display the results.
center = redwoods.centroid().getInfo()['coordinates']
center.reverse()
dicc ={'NAIP input imagery': naip.getMapId(),
       'NDVI': naipNDVI.getMapId({'min': -1, 'max': 1, 'palette': ['FF0000', '00FF00']}),
       'SD of NDVI':texture.getMapId({'min': 0, 'max': 0.3})}
Mapdisplay(center,dicc,zoom_start=12)

# Statistics of FeatureCollection Columns

To reduce properties of features in a `FeatureCollection`, use `featureCollection.reduceColumns()`. Consider the following toy example:


In [ ]:
# Make a toy FeatureCollection.
aFeatureCollection = ee.FeatureCollection([
  ee.Feature(None, {'foo': 1, 'weight': 1}),
  ee.Feature(None, {'foo': 2, 'weight': 2}),
  ee.Feature(None, {'foo': 3, 'weight': 3}),
])

# Compute a weighted mean and display it.
pprint(aFeatureCollection.reduceColumns(**{
  'reducer': ee.Reducer.mean(),
  'selectors': ['foo'],
  'weightSelectors': ['weight']
}).getInfo())

As a more complex example, consider a `FeatureCollection` of US counties with census data as attributes. The variables of interest are total population and total housing units. You can get their sum(s) by supplying a summing reducer argument to `reduceColumns()` and printing the result:

In [ ]:
# Load a collection of US counties with census data properties and display it.
counties = ee.FeatureCollection('ft:1S4EB6319wWW2sWQDPhDvmSBIVrD3iEmCLYB7nMM')

# Compute sums of the specified properties and print the resultant Dictionary.
sums = counties.filter(ee.Filter.And(ee.Filter.neq('Census 2000 Population', None),
                                     ee.Filter.neq('Census 2000 Housing Units', None)))\
               .reduceColumns(**{'reducer': ee.Reducer.sum().repeat(2),
                               'selectors': ['Census 2000 Population', 'Census 2000 Housing Units']})
print(sums.getInfo())

# Display with folium!
center = counties.geometry().centroid().getInfo()['coordinates']
center.reverse()
Mapdisplay(center,{'Census':counties.getMapId()},zoom_start=5)

Note that because feature collections may contain missing data (unlike images, which handle missing data with masks), the input needs to be pre-filtered to eliminate null values.

An error that looks something like the following may be thrown as a result of attributes with **None** values:

```
Dictionary (Error)
  Collection.reduceColumns: Can't set input 0 of Reducer(reducer=SUM, count=2) to .:
  Input must be a scalar number.
```  
Also note that unlike `imageCollection.reduce()`, in which reducers are automatically repeated for each band, reducers on a `FeatureCollection` must be explicitly repeated using `repeat()`. Specifically, repeat the reducer m times for m inputs. The following error may be thrown as a result of not repeating the reducer:

```
Dictionary (Error)
  Collection.reduceColumns: Need 1 inputs for <Reducer>, got 2.
```



# Raster to Vector Conversion

To convert from an `Image` (raster) to a `FeatureCollection` (vector) data type, use `image.reduceToVectors()`. This is the primary mechanism for vectorization in Earth Engine, and can be useful for generating regions for input to other types of reducer. The `reduceToVectors()` method creates polygon edges (optionally centroids or bounding boxes instead) at the boundary of homogeneous groups of connected pixels.

For example, consider a 2012 nightlights image of Japan. Let the nightlights digital number serve as a proxy for development intensity. Define zones using arbitrary thresholds on the nightlights, combine the zones into a single-band image, vectorize the zones using `reduceToVectors()`:

In [ ]:
# Load a Japan boundary from the Large Scale International Boundary dataset.
japan = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')\
          .filter(ee.Filter.eq('country_na', 'Japan'))

# Load a 2012 nightlights image, clipped to the Japan border.
nl2012 = ee.Image('NOAA/DMSP-OLS/NIGHTTIME_LIGHTS/F182012')\
           .select('stable_lights')\
           .clipToCollection(japan)

# Define arbitrary thresholds on the 6-bit nightlights image.
zones = nl2012.gt(30)\
              .add(nl2012.gt(55))\
              .add(nl2012.gt(62))\
              .updateMask(zones.neq(0))


# Convert the zones of the thresholded nightlights to vectors.
vectors = zones.addBands(nl2012).reduceToVectors(**{
  'geometry': japan,
  'crs': nl2012.projection(),
  'scale': 1000,
  'geometryType': 'polygon',
  'eightConnected': False,
  'labelProperty': 'zone',
  'reducer': ee.Reducer.mean()
})

display = ee.Image(0).updateMask(0).paint(vectors, '000000', 3)


# Display the thresholds.
center = [35.712, 139.6225]
zones_tk = zones.getMapId({'min': 1, 'max': 3, 'palette': ['0000FF', '00FF00', 'FF0000']})
display_tk = display.getMapId({'palette': '000000'})

Mapdisplay(center,{'raster':zones_tk,'vectors':display_tk},zoom_start=9)


Note that the first band in the input is used to identify homogeneous regions and the remaining bands are reduced according to the provided reducer, the output of which is added as a property to the resultant vectors. The `geometry` parameter specifies the extent over which the vectors should be created. In general, it is good practice to specify a minimal zone over which to create vectors. It is also good practice to specify the `scale` and `crs` to avoid ambiguity. The output type is `‘polygon’` where the polygons are formed from homogeneous zones of four-connected neighbors (i.e. `eightConnected` is false). The last two parameters, `labelProperty` and reducer, specify that the output polygons should receive a property with the zone label and the mean of the nightlights band(s), respectively.

# Vector to Raster Conversion

Vector to raster conversion in Earth Engine is handled by the `featureCollection.reduceToImage()` method. This method assigns pixels under each feature the value of the specified property. This example uses the counties data to create an image representing the population of each county:

In [ ]:
# Load a collection of US counties with census data properties.
counties = ee.FeatureCollection('ft:1S4EB6319wWW2sWQDPhDvmSBIVrD3iEmCLYB7nMM')

# Make an image out of the population attribute and display it.
popImage = counties.filter(ee.Filter.neq('Census 2000 Population', None))\
                   .reduceToImage(**{'properties': ['Census 2000 Population'],
                                     'reducer': ee.Reducer.first()})
center = [40.38, -99.976]
popImage_tk = popImage.getMapId({'min': 0, 'max': 1000000,
                                 'palette': ['0000FF', '00FF00', '00FFFF', 'FF0000']})
Mapdisplay(center, {'popImage':popImage_tk}, zoom_start=5)

If the features overlap, specify a reducer to indicate how to aggregate properties of overlapping features. In the previous example, since there is no overlap, an ee.Reducer.first() is sufficient. As in this example, pre-filter the data to eliminate nulls that can not be turned into an image. 

# Grouped Reductions and Zonal Statistics

You can get statistics in each zone of an `Image` or `FeatureCollection` by using `reducer.group()` to group the output of a reducer by the value of a specified input. For example, to compute the total population and number of housing units in each state, this example groups the output of a reduction of a counties `FeatureCollection` as follows:




In [ ]:
# Load a collection of US counties with census data properties.
counties = ee.FeatureCollection('ft:1S4EB6319wWW2sWQDPhDvmSBIVrD3iEmCLYB7nMM')

# Compute sums of the specified properties, grouped by state name.
sums = counties.filter(ee.Filter.And(ee.Filter.neq('Census 2000 Population', None),
                                     ee.Filter.neq('Census 2000 Housing Units', None)))\
               .reduceColumns(**{
                  'selectors': ['Census 2000 Population', 'Census 2000 Housing Units', 'StateName'],
                  'reducer': ee.Reducer.sum().repeat(2).group(**{
                      'groupField': 2,
                      'groupName': 'state'})})

# Print the resultant Dictionary.
pprint(sums.getInfo())

The `groupField` argument is the index of the input in the selectors array that contains the codes by which to group, the `groupName` argument specifies the name of the property to store the value of the grouping variable. Since the reducer is not automatically repeated for each input, the `repeat(2)` call is needed.

To group output of `image.reduceRegions()` you can specify a grouping band that defines groups by integer pixel values. This type of computation is sometimes called "zonal statistics" where the zones are specified as the grouping band and the statistic is determined by the reducer. In the following example, change in nightlights in the United States is grouped by land cover category:

In [ ]:
# Load a region representing the United States
region = ee.Feature(ee.FeatureCollection('ft:1tdSwUL7MVpOauSgRzqVTOwdfy17KDbw-1d9omPw')\
                      .filter(ee.Filter.eq('Country', 'United States'))\
                      .first())

# Load MODIS land cover categories in 2001.
landcover = ee.Image('MODIS/051/MCD12Q1/2001_01_01')\
              .select('Land_Cover_Type_1') # Select the IGBP classification band.

# Load nightlights image inputs.
nl2001 = ee.Image('NOAA/DMSP-OLS/NIGHTTIME_LIGHTS/F152001')\
           .select('stable_lights')

nl2012 = ee.Image('NOAA/DMSP-OLS/NIGHTTIME_LIGHTS/F182012')\
           .select('stable_lights')

# Compute the nightlights decadal difference, add land cover codes.
nlDiff = nl2012.subtract(nl2001).addBands(landcover)

# Grouped a mean reducer: change of nightlights by land cover category.
means = nlDiff.reduceRegion(**{
  'reducer': ee.Reducer.mean().group(**{
    'groupField': 1,
    'groupName': 'code',
  }),
  'geometry': region.geometry(),
  'scale': 1000,
  'maxPixels': 1e8
});

# Print the resultant Dictionary.
pprint(means.getInfo())

Note that in this example, the `groupField` is the index of the band containing the zones by which to group the output. The first band is index 0, the second is index 1, etc.